# Output Parser
Format the output to post-processing the data, example: receive data from LLM and use in the system, after that return to the user.

The parsers padronize the outputs.

### Converting response to JSON:

In [27]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import ChatPromptTemplate

client_review = """This leaf blower is pretty awesome. It has four settings: candle blow, gentle breeze, 
windy city, and tornado. It arrived in two days, just in time for my wife's birthday present. I think my 
wife loved it so much she was speechless. So far, I've been the only one using it, and I've been using it 
every other morning to clear the leaves from our lawn. It's a little more expensive than other leaf blowers 
on the market, but I think it's worth it for the extra features."""

present_schema = ResponseSchema(
    name='is_present',
    type='boolean',
    description='Whether the product is a present or gift.'
)

schema_delivery = ResponseSchema(
    name='delivery_days',
    type='int',
    description='Number of days it took for the product to be delivered.'
)

perception_value = ResponseSchema(
    name='perception_value',
    type='list',
    description='Extract any phrase about the perception of value about the product. Return as a Python list.'
)

response_schema = [present_schema, schema_delivery, perception_value]

output_parser = StructuredOutputParser.from_response_schemas(response_schema)

schema_format_instructions = output_parser.get_format_instructions()

In [28]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

review_template = ChatPromptTemplate.from_template("""
Given the text, extract the following information:                                                   

is_present, delivery_days and perception_value

Text: {review}

{schema}

""", partial_variables={'schema': schema_format_instructions})

prompt = review_template.format_messages(review=client_review)

response = llm.invoke(prompt)
print(response.content)

```json
{
	"is_present": true,
	"delivery_days": 2,
	"perception_value": ["I think it's worth it for the extra features"]
}
```


### Converting response to JSON:

In [29]:
json_response = output_parser.parse(response.content)

print(json_response['is_present'])

True
